In [36]:
from tools import *
%matplotlib notebook
import pandas as pd

from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle

from dustmaps.bayestar import BayestarQuery

data = pd.read_csv('K2_all_PS1_psc_ev_crowding_sdss.csv')

In [34]:
data = data.iloc[data.campaign.values == 4]


In [37]:
len(data)

104204

In [7]:
gr = (data.gMeanPSFMag - data.rMeanPSFMag).values
data = data.iloc[gr<1]

In [9]:
from tqdm import tqdm

In [10]:
catalog = "I/347/gaia2dis"
c1 = SkyCoord(data['RAJ2000'].values, data['DecJ2000'].values, frame='icrs', unit='deg')
d = []
d_low = []
d_high = []
for i in tqdm(range(len(c1))):
    result = Vizier.query_region(c1[i], catalog=[catalog],radius=Angle(3, "arcsec"))
    if len(result) != 0:
        #print('beep')
        r = result[catalog].to_pandas()
        #if len(r) > 1:
        r = r.iloc[0]
        d += [r['rest']]
        d_low += [r['b_rest']]
        d_high += [r['B_rest']]
    else:
        d += [np.nan]
        d_low += [np.nan]
        d_high += [np.nan]



100%|██████████| 583/583 [13:53<00:00,  1.43s/it]


In [11]:
d

[395.083601872126,
 268.928985755884,
 733.535557278599,
 298.137432106142,
 555.630210279968,
 976.336271351922,
 832.306020654105,
 1447.02790479063,
 514.626563752702,
 646.841709681328,
 1124.05529779852,
 479.260807436498,
 332.744204760393,
 1259.82050029107,
 869.148269093369,
 329.106625139281,
 445.162994529886,
 438.945495367854,
 178.377429507848,
 1057.52314978974,
 1496.1896791912,
 1458.67906170557,
 291.953194154419,
 307.235236689283,
 977.13847694704,
 1642.80972709585,
 373.600213589242,
 609.156447977633,
 1594.54382717721,
 434.365904307363,
 241.260649594386,
 1703.96139892277,
 489.270137374985,
 1513.74681543273,
 521.972992516253,
 374.777679786223,
 281.192110932921,
 250.157424040732,
 1332.03727489936,
 313.893309440014,
 521.231764066025,
 938.01209194836,
 1022.27186630484,
 724.142722173493,
 248.105905132256,
 128.423985033567,
 1266.33036944234,
 168.947657122076,
 387.803396723837,
 949.501269846435,
 573.358432976047,
 113.826266629504,
 610.0786587465

In [21]:

coords = SkyCoord(c1.ra.degree*u.deg, c1.dec.degree*u.deg,
                  distance=d*u.pc, frame='icrs')
bayestar = BayestarQuery()
coords = SkyCoord(c1.ra.degree*u.deg, c1.dec.degree*u.deg,
                  distance=d*u.pc, frame='icrs')
med = bayestar(coords, mode='median')

coords = SkyCoord(c1.ra.degree*u.deg, c1.dec.degree*u.deg,
                  distance=d_low*u.pc, frame='icrs')
low = bayestar(coords, mode='median')

coords = SkyCoord(c1.ra.degree*u.deg, c1.dec.degree*u.deg,
                  distance=d_high*u.pc, frame='icrs')
high = bayestar(coords, mode='median')


Loading pixel_info ...
Loading samples ...
Loading best_fit ...
Replacing NaNs in reliable distance estimates ...
Sorting pixel_info ...
Extracting hp_idx_sorted and data_idx at each nside ...
  nside = 64
  nside = 128
  nside = 256
  nside = 512
  nside = 1024
t = 45.514 s
  pix_info:   0.569 s
   samples:  30.429 s
      best:   6.021 s
       nan:   0.258 s
      sort:   8.013 s
       idx:   0.223 s


In [26]:
(med-high) * 3.518 

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.0036487 ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.00217982,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.04518675,  0.        ,  0.        ,
       -0.02589021,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [29]:
a= np.array([low,med,high])

In [33]:
np.save('c4_ext.npy',a)

In [38]:
from astropy.io import fits

In [40]:
c4 = data.iloc[data['campaign'].values ==4 ]

In [46]:
a = c4[['RAJ2000','DecJ2000']].iloc[:100]

In [47]:
a.to_csv('../c4pos_small.csv',index=False)

In [48]:
a = fits.open('../../gaia_dist.fit')

In [60]:
a[0].header

SIMPLE  =                    T / Standard FITS Format                           
BITPIX  =                    8 / Character data                                 
NAXIS   =                    0 / No Image --- just extension(s)                 
EXTEND  =                    T / There are standard extensions                  
ORIGIN  = 'xml2fits_v1.95'     / Converted from XML-Astrores to FITS            
                         e-mail: question@simbad.u-strasbg.fr                   
LONGSTRN= 'OGIP 1.0'           / Long string convention (&/CONTINUE) may be used
DATE    = '2021-01-21'         / Written on 2021-01-21:02:37:30 (GMT)           
                             by: apache@vizier.u-strasbg.fr                     
         **********************************************************             
             EXCERPT from catalogues stored in VizieR (CDS)                     
                        with the following conditions:                          
         *******************

In [62]:
from astropy.table import Table

In [66]:
table = Table.read('../../gaia_dist.fit',format='ascii')

KeyboardInterrupt: 

In [86]:
table = Table.read('../../data/c4_test.fit', hdu=10).to_pandas()

In [88]:
table.append(table,ignore_index=True)

,Source,RA_ICRS,DE_ICRS,rest,b_rest,b_rest_lc,rlen,ResFlag,ModFlag
0,2245632,59.010382,11.368485,733.535557,682.057124,793.186242,391.683163,b'1',1
1,2245632,59.010382,11.368485,733.535557,682.057124,793.186242,391.683163,b'1',1


In [79]:
f = fits.open('../../gaia_dist.fit')

In [80]:
len(f)

159619

In [82]:
table = Table.read('../../gaia_dist.fit',hdu=159619-1)

In [ ]:
159619

In [83]:
table.

Source,RA_ICRS,DE_ICRS,rest,b_rest,b_rest_lc,rlen,ResFlag,ModFlag
,deg,deg,pc,pc,pc,pc,,
int32,float64,float64,float64,float64,float64,float64,bytes1,int32
3245184,101.35599065314,28.96144876859,351.10038897404,347.989289734425,354.266606345288,717.966792383435,1,1


In [89]:
from tqdm import tqdm

In [ ]:

for i in range(159619-1):
    if i == 0:
        gaia = Table.read('../../gaia_dist.fit',hdu=i).to_pandas()
        if len(gaia) > 1:
            gaia = gaia.iloc[0]
    else:
        table = Table.read('../../gaia_dist.fit',hdu=i).to_pandas()
        if len(table) > 1:
            table = table.iloc[0]
        gaia = gaia.append(table,ignore_index=True)